In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
#from vsopy import util, data
import astropy.units as u
from SetupSession import use_dark_theme


In [ ]:
import vso.util
import vso.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='2025/20250126'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
IMAGE_DIR = vso.util.ImageLayout(IMAGE_ROOT).get_images(session).lights_dir / 'V'
sd = vso.data.StarData(layout.charts_dir)

In [ ]:
import ccdproc as ccdp
import vso.reduce

ifc = ccdp.ImageFileCollection(IMAGE_DIR)
path = IMAGE_DIR / ifc.summary['file'][0]
image  = vso.reduce.load_and_solve(path, session_layout.solved_dir)
matcher = vso.reduce.CalibrationMatcher(layout.calibr_dir, temp_tolerance=2*u.K)
cal = matcher.match(image.header)
reduced = vso.reduce.calibrate_image(image,
                        dark=cal.dark,
                        flat=cal.flat)


In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.coordinates import SkyCoord
import vso.phot
from astropy.table import QTable
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon, Ellipse


In [ ]:

mean, median, std = sigma_clipped_stats(reduced.data, sigma=3.0)
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std)

sources = daofind(reduced.data)
sources

In [ ]:
import vso.phot
from astropy.table import QTable

stars = QTable(dict(radec2000=reduced.wcs.pixel_to_world(sources['xcentroid'], sources['ycentroid']),
                    auid=sources['id']))

stars = vso.phot.measure_photometry(reduced, stars, vso.util.Aperture(5, 10, 15))
#stars.rename_columns(['sky_centroid'], ['radec2000'])
stars[stars['snr'].value > 5]

In [ ]:
from astroquery.vizier import Vizier

center = SkyCoord(ra = reduced.wcs.wcs.crval[0] *u.deg, dec= reduced.wcs.wcs.crval[1] *u.deg)

vizier = Vizier(columns=['NOMAD1', 'Tycho-2', 'RAJ2000', 'DEJ2000', 'Bmag', 'Vmag', 'e_Bmag', 'e_Vmag', 'Rmag', 'e_RAJ2000', 'e_DEJ2000', 'pmRA', 'pmDE'])
# vizier = Vizier(columns=['DR3Name', 'RA_ICRS', 'DE_ICRS', 'Gmag', 'BPmag', 'RPmag'])
vizier.ROW_LIMIT = -1
result = vizier.query_region(center,
                             width=55*u.arcmin,
                             height=55*u.arcmin,
                             #catalog='II/336/apass9',
                             #catalog='I/305',               # GSC 2.3
                             catalog='I/297',               # NOMAD
                            #  catalog='I/355/gaiadr3',       # Gaia DR3
                            #  catalog='I/239/tyc_main',               # Hipparcos
                             column_filters={'Vmag': '<16'})
cat_result = result[0]

# cat_result['radec2000'] = SkyCoord(ra=cat_result['RA_ICRS'], # Gaia
#                                    dec=cat_result['DE_ICRS'])
# cat_result.remove_columns(['RA_ICRS', 'DE_ICRS'])
cat_result['radec2000'] = SkyCoord(ra=cat_result['RAJ2000'], # GSC, NOMAD
                                   dec=cat_result['DEJ2000'])
cat_result.remove_columns(['RAJ2000', 'DEJ2000'])
# cat_result['radec2000'] = SkyCoord(ra=cat_result['RAICRS'], # Hipparcos
#                                    dec=cat_result['DEICRS'],
#                                    frame='icrs')
# cat_result.remove_columns(['RAICRS', 'DEICRS'])

cat_result

In [ ]:
vizier = Vizier(columns=['GCVS', 'VarType', 'RAJ2000', 'DEJ2000', 'magMax', 'Min1', 'Min2', 'n_Min1', 'n_Min2', 'flt', 'Period'])
#vizier = Vizier(columns=['GCVS', 'Vmag', 'RAJ2000', 'DEJ2000'])
vizier.ROW_LIMIT = -1
result = vizier.query_region(center,
                             width=55*u.arcmin,
                             height=55*u.arcmin,
                             catalog='B/gcvs',               # GCVS
                             column_filters={})
vars = result[0]

vars['radec2000'] = SkyCoord(ra=vars['RAJ2000'], # GSC, NOMAD
                                   dec=vars['DEJ2000'], unit=(u.hourangle, u.deg))
vars.remove_columns(['RAJ2000', 'DEJ2000'])
vars

In [ ]:
vizier = Vizier()#columns=['GCVS', 'VarType', 'RAJ2000', 'DEJ2000', 'magMax', 'Min1', 'Min2', 'n_Min1', 'n_Min2', 'flt', 'Period'])
vizier.ROW_LIMIT = -1
result = vizier.query_region(center,
                             width=55*u.arcmin,
                             height=55*u.arcmin,
                             catalog='VII/237/pgc',               # PGC/HYPERLEDA
                             column_filters={})
gals = result[0]

gals['radec2000'] = SkyCoord(ra=gals['RAJ2000'], # GSC, NOMAD
                                   dec=gals['DEJ2000'], unit=(u.hourangle, u.deg))
# gals.remove_columns(['RAJ2000', 'DEJ2000'])
gals

In [ ]:
centroids = QTable.read(session_layout.centroid_file_path)
# cat_stars = cat_result[cat_result['Class'] == 0]
# cat_gals = cat_result[cat_result['Class'] == 1]
# image.header['DATE-OBS']
cat_stars = cat_result
cat_stars['pmRA'] * 25 *u.mas

In [ ]:
%matplotlib ipympl
from astropy.visualization import AsinhStretch, LogStretch, ImageNormalize, MinMaxInterval
from matplotlib import cm

cmap = cm.get_cmap('gray').reversed()
snr_max = np.max(stars['snr'].value)

t_o = 25*u.mas

alpha = 0.001
interval = MinMaxInterval()
fig = plt.figure(figsize=(16.00, 16.00))
ax = plt.subplot(projection=reduced.wcs)
vmin, vmax = interval.get_limits(reduced.data)
norm = ImageNormalize(vmin=mean+2*std, vmax=vmax, stretch=LogStretch(10000))
ax.imshow(reduced.data, origin='lower', norm=norm, cmap=cmap)
for t in stars:
    r = 5 # 2*(20 - t['M']['mag'].value)
    c = reduced.wcs.world_to_pixel(t['radec2000'])
    ax.add_patch(Circle(c, r, color='red', alpha=.8, fill=False)) #alpha=np.clip(t['snr'].value/snr_max, 0, .9)
for s in cat_stars:
    r = 15 #2*(20 - s['Vmag'])
    c = reduced.wcs.world_to_pixel(s['radec2000'])
    ax.add_patch(Circle(c, r, color='blue', alpha=.8, fill=False))
    # c1 = reduced.wcs.world_to_pixel(s['radec2000'].spherical_offsets_by(s['pmRA']*t_o, s['pmDE']*t_o))
    # ax.add_patch(Circle(c1, 10, color='blue', alpha=.5, fill=False))
for s in vars:
    r = 40
    c = reduced.wcs.world_to_pixel(s['radec2000'])
    ax.add_patch(RegularPolygon(c, 4, radius=r, color='orange', fill=False, alpha=.8))
for comp in centroids:
    r = 40
    c = reduced.wcs.world_to_pixel(comp['radec2000'])
    ax.add_patch(RegularPolygon(c, 3, radius=r, color='gray', fill=False, alpha=.8))
for g in gals:
    r = 20
    c = reduced.wcs.world_to_pixel(g['radec2000'])
    ax.add_patch(Ellipse(c, r, r/2, angle=g['PA'], color='gray', fill=False, alpha=.8))
# for s in cat_gals:
#     r = 2*(20 - s['Vmag'])
#     c = reduced.wcs.world_to_pixel(s['radec2000'])
#     ax.add_patch(Circle(c, r, color='yellow', alpha=.5))

plt.grid(color='black', ls='dashed')
plt.show()


In [ ]:

# cat_stars = cat_result[cat_result['Class'] == 0]
# cat_gals = cat_result[cat_result['Class'] == 1]
# image.header['DATE-OBS']
cat_stars = cat_result

In [ ]:
from astropy.table import join, join_skycoord


confirmed = join(stars, cat_stars, keys='radec2000', join_funcs={'radec2000': join_skycoord(2 * u.arcsec)})
confirmed = confirmed[confirmed['snr'].value > 13]
confirmed['radec_err'] = confirmed['radec2000_1'].separation(confirmed['radec2000_2']).to(u.arcsec)
confirmed

In [ ]:
import scipy.stats as sst

rr = sst.linregress(confirmed['M']['mag'].value, confirmed['Vmag'])
rr_slope_err2 = (rr.stderr/rr.slope) ** 2

def pred(x):
    return rr.intercept*u.mag + rr.slope*x

def pred_err(x, err):
    x_err2 = (err/x).value**2
    rr_err2 = np.full((len(x),), rr_slope_err2)
    return np.sqrt((rr.intercept_stderr)**2 + (x*rr.slope).value * np.sqrt(x_err2 + rr_err2))*u.mag

predicted = pred(confirmed['M']['mag']) #rr.intercept*u.mag + rr.slope*confirmed['V']
err = pred_err(confirmed['M']['mag'], confirmed['M']['err'])
good = np.abs(predicted - confirmed['Vmag']) < 1*err
rr


In [ ]:


plt.plot(confirmed['M']['mag'][good].value, confirmed['Vmag'][good].value, '.')
plt.plot(confirmed['M']['mag'][~good], confirmed['Vmag'][~good].value, 'o')
plt.plot(confirmed['M']['mag'][good], predicted[good], 'r')
plt.plot(confirmed['M']['mag'][good], (predicted+1*err)[good], 'g-')
plt.plot(confirmed['M']['mag'][good], (predicted-1*err)[good], 'g-')

# plt.legend()
plt.title(fr'$ R^2 = {rr.rvalue**2:.3g}$')
plt.show()

In [ ]:
confirmed[~good]